In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
r = requests.get("https://www.lsa.umich.edu/cg/cg_subjectlist.aspx?termArray=f_20_2310&cgtype=ug&allsections=true")

In [4]:
soup = BeautifulSoup(r.text)

In [11]:
departments = [cell.text.strip() for cell in soup.select_one(".table.table-striped.table-condensed").select("tr > td:nth-child(1)")]

In [122]:
ug_course_sections = []

In [121]:
len(soup.select(".row.result") + soup.select(".row.resultalt"))

4

In [123]:
url = "https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=f_20_2310&cgtype=ug&department={}&allsections=true&show=500"
for dep in departments:
    print(dep)
    r = requests.get(url.format(dep))
    soup = BeautifulSoup(r.text)
    for result in soup.select(".row.result") + soup.select(".row.resultalt"):
        dept, number, _, name = [a.strip() for a in result.select_one("font").text.strip().split("\r\n")]
        section, term, credits, mode, instructor, _ = [a.text.strip() for a in result.select_one(".bottompadding_main").find_all("div", recursive=False)]
        section = " ".join(section.split())
        term = " ".join(term.split()[1:])
        credits = credits.split()[-1]
        mode = mode.split()[-1]
        instructor = " ".join(instructor.split()[1:])
        ug_course_sections.append({
            "dept": dept,
            "number": number,
            "name": name,
            "section": section,
            "term": term,
            "credits": credits,
            "mode": mode,
            "instructor": instructor,
        })

AAS
AERO
AEROSP
AES
ALA
AMCULT
ANATOMY
ANTHRARC
ANTHRBIO
ANTHRCUL
APPPHYS
ARABAM
ARABIC
ARCH
ARMENIAN
ARTDES
ARTSADMN
ASIAN
ASIANLAN
ASIANPAM
ASTRO
AUTO
BA
BCS
BIOINF
BIOLCHEM
BIOLOGY
BIOMEDE
BIOPHYS
BIOSTAT
CATALAN
CEE
CHE
CHEM
CJS
CLARCH
CLCIV
CLIMATE
CMPLXSYS
COGSCI
COMM
COMP
COMPLIT
CSP
CZECH
DANCE
DATASCI
DIGITAL
DUTCH
EARTH
EAS
ECON
EDCURINS
EDUC
EEB
EECS
ELI
ENGLISH
ENGR
ENS
ENSCEN
ENVIRON
ES
ESENG
FRENCH
FTVM
GEOG
GERMAN
GREEK
GREEKMOD
GTBOOKS
HEBREW
HISTART
HISTORY
HONORS
HS
HUMGEN
INSTHUM
INTLSTD
INTMED
IOE
ISLAM
ITALIAN
JAZZ
JUDAIC
KINESLGY
KRSTD
LACS
LATIN
LATINOAM
LING
LSWA
MACROMOL
MATH
MATSCIE
MCDB
MECHENG
MEDCHEM
MELANG
MEMS
MENAS
MFG
MICRBIOL
MIDEAST
MILSCI
MKT
MOVESCI
MUSEUMS
MUSICOL
MUSMETH
MUSTHTRE
NATIVEAM
NAVARCH
NAVSCI
NERS
NURS
ORGSTUDY
PAT
PATH
PERSIAN
PHARMACY
PHARMSCI
PHIL
PHRMACOL
PHYSICS
PHYSIOL
PIBS
POLISH
POLSCI
PORTUG
PPE
PSYCH
PUBHLTH
PUBPOL
QMSS
RCARTS
RCASL
RCCORE
RCHUMS
RCIDIV
RCLANG
RCMUSIC
RCNSCI
RCSSCI
REEES
RELIGION
ROMLANG
ROMLING
RUSSIAN
SCAND


In [124]:
len(ug_course_sections)

4743

In [92]:
import pandas as pd

In [125]:
df = pd.DataFrame(ug_course_sections)

In [126]:
df["mode"].value_counts() / df["mode"].count()

Online    0.643896
Person    0.226861
Hybrid    0.129243
Name: mode, dtype: float64

In [127]:
df[(df.dept == "EECS") & (df.number == "376")]

,dept,number,name,section,term,credits,mode,instructor
1288,EECS,376,Foundations of Computer Science,Section 002 (LEC),FA 2020,4,Online,"Kamil,Amir"
1289,EECS,376,Foundations of Computer Science,Section 004 (LEC),FA 2020,4,Online,"Volkovich,Ilya"
1372,EECS,376,Foundations of Computer Science,Section 001 (LEC),FA 2020,4,Online,"Kamil,Amir"
1373,EECS,376,Foundations of Computer Science,Section 003 (LEC),FA 2020,4,Online,"Volkovich,Ilya"


In [128]:
df.groupby("dept")["mode"].value_counts() / df.groupby("dept")["mode"].count()

dept     mode  
AAS      Online    0.442857
         Person    0.328571
         Hybrid    0.228571
AERO     Hybrid    0.750000
         Online    0.250000
                     ...   
WGS      Hybrid    0.083333
WRITING  Online    0.833333
         Hybrid    0.166667
YIDDISH  Online    0.500000
         Person    0.500000
Name: mode, Length: 402, dtype: float64

In [129]:
df.groupby("mode").get_group("Person").dept.value_counts()[:10]

THTREMUS    137
STDABRD      86
PSYCH        58
ENVIRON      49
ENGLISH      34
GERMAN       25
ARTDES       25
CHEM         23
AAS          23
BIOLOGY      20
Name: dept, dtype: int64

In [108]:
df.groupby("mode").get_group("Hybrid").dept.value_counts()[:10]

ENGLISH     22
MATH        15
AAS         14
FTVM        13
MECHENG     11
HISTORY     10
ASIANLAN    10
RCLANG      10
EECS         9
EARTH        8
Name: dept, dtype: int64